In [ ]:

import pandas as pd
from keybert import KeyBERT
import spacy
from collections import Counter
from string import punctuation

def getTextList(fileName, column):
    df = pd.read_csv(fileName)
    return df[column]

def getKeywords_keyBERT(text):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(text)
    return keywords


def getkeywords_Spacy(text):
    nlp = spacy.load("en_core_web_sm")

    def get_hotwords(text):
        result = []
        pos_tag = ['PROPN', 'ADJ', 'NOUN']
        doc = nlp(text.lower())
        for token in doc:
            if (token.text in nlp.Defaults.stop_words or token.text in punctuation):
                continue
            if (token.pos_ in pos_tag):
                result.append(token.text)
        return result

    output = set(get_hotwords(text))
    most_common_list = Counter(output).most_common(10)
    return most_common_list

def insertcolumn(fileName, column, data):
    df = pd.read_csv(fileName)
    df[column] = data
    df.to_csv(fileName)

if __name__ == '__main__':
    files = ["BlinkMini_cleansentiment.csv", "BlinkMiniSecurity_cleansentiment.csv", "BlinkVideoSecurity_cleansentiment.csv", "echo_show_cleansentiment.csv", "mcAf_cleansentiment.csv", "NooieBabyMonitor_cleansentiment.csv", "ringSpotLight_cleansentiment.csv"]
    for fileName in files:
        reviews = getTextList(fileName, "Total Description")
        keywords = list()
        for i in range(reviews.size):
            review = reviews.iloc[i]
            if not isinstance(review, str) or len(review) == 0:
                keywords.append([])
                continue
            keys2 = getkeywords_Spacy(review)
            keywords.append([i[0] for i in keys2 if len(i[0])>2])
            print(str(i) + " done ")
        insertcolumn(fileName, "keywords", keywords)
        print(fileName + " done ")

